# Use case 1: Global Navigation Satelite System (GNSS) Positioning.
This notebook tests the efficacy of Bicomplex Neural Networks in GNSS positioning and contrasts this performance with existing frameworks.





## 0: Data loading

### 0.1: Libraries

In [ ]:
from datetime import datetime, timezone
import pandas as pd
import numpy as np
import pynmea2
import os

from networkx.algorithms.flow import network_simplex

### 0.2: Data extraction

In [1]:
# --- 1. Load the GnssLog.txt file ---
log_path = 'Pixel4XL_GnssLog.txt'

# First, let's read the header line to get column names for Raw measurements
header_line = ""
with open(log_path, 'r') as f:
    for line in f:
        if line.strip().startswith('# Raw'):
            header_line = line.strip()
            break

# Extract column names
columns = header_line.replace('# ', '').split(',')
print(f"Found {len(columns)} columns in Raw header")

# --- 2. Read ONLY the 'Raw' lines ---
# We'll manually parse the file to avoid the field count mismatch
raw_rows = []
with open(log_path, 'r') as f:
    for line in f:
        # Skip comment lines
        if line.startswith('#'):
            continue
        # Only keep lines that start with 'Raw'
        if line.strip().startswith('Raw'):
            raw_rows.append(line.strip().split(','))

# Create DataFrame from raw rows
df_log = pd.DataFrame(raw_rows, columns=columns)

# --- 3. Select the Columns We Need ---
# Convert to numeric types
num_cols = [
    'TimeNanos', 'Svid', 'Cn0DbHz',
    'AccumulatedDeltaRangeMeters', 'CarrierFrequencyHz',
    'ConstellationType'
]

for col in num_cols:
    if col in df_log.columns:
        df_log[col] = pd.to_numeric(df_log[col], errors='coerce')

# --- 4. Filter for a Constellation (e.g., GPS) ---
# ConstellationType 1 is GPS
gps_data = df_log[df_log['ConstellationType'] == 1].copy()

# Remove any rows with missing critical values
gps_data = gps_data.dropna(subset=['TimeNanos', 'Svid', 'CarrierFrequencyHz'])

Found 37 columns in Raw header

Loaded 50196 'Raw' measurements total.
Found 16546 GPS measurements.

Here are the key columns you will use:
       TimeNanos  Svid  CarrierFrequencyHz  Cn0DbHz  \
0  4514908000000     5        1.575420e+09     31.4   
1  4514908000000    10        1.575420e+09     19.2   
2  4514908000000    13        1.575420e+09     31.6   
3  4514908000000    15        1.575420e+09     37.1   
4  4514908000000    16        1.575420e+09     25.0   
5  4514908000000    18        1.575420e+09     36.6   
6  4514908000000    20        1.575420e+09     33.6   
7  4514908000000    25        1.575420e+09     36.2   
8  4514908000000    26        1.575420e+09     27.6   
9  4514908000000    29        1.575420e+09     39.4   

   AccumulatedDeltaRangeMeters  
0                 1.276232e+06  
1                 0.000000e+00  
2                -5.477529e+04  
3                -2.522253e+05  
4                -3.787790e+04  
5                -7.384776e+05  
6                -3.25

In [2]:
list(gps_data.columns)

['Raw',
 'utcTimeMillis',
 'TimeNanos',
 'LeapSecond',
 'TimeUncertaintyNanos',
 'FullBiasNanos',
 'BiasNanos',
 'BiasUncertaintyNanos',
 'DriftNanosPerSecond',
 'DriftUncertaintyNanosPerSecond',
 'HardwareClockDiscontinuityCount',
 'Svid',
 'TimeOffsetNanos',
 'State',
 'ReceivedSvTimeNanos',
 'ReceivedSvTimeUncertaintyNanos',
 'Cn0DbHz',
 'PseudorangeRateMetersPerSecond',
 'PseudorangeRateUncertaintyMetersPerSecond',
 'AccumulatedDeltaRangeState',
 'AccumulatedDeltaRangeMeters',
 'AccumulatedDeltaRangeUncertaintyMeters',
 'CarrierFrequencyHz',
 'CarrierCycles',
 'CarrierPhase',
 'CarrierPhaseUncertainty',
 'MultipathIndicator',
 'SnrInDb',
 'ConstellationType',
 'AgcDb',
 'BasebandCn0DbHz',
 'FullInterSignalBiasNanos',
 'FullInterSignalBiasUncertaintyNanos',
 'SatelliteInterSignalBiasNanos',
 'SatelliteInterSignalBiasUncertaintyNanos',
 'CodeType',
 'ChipsetElapsedRealtimeNanos']

In [3]:
# Use the SPAN file for ground truth, or the chipset file for baseline
nmea_file_path = 'SPAN_Pixel4XL_10Hz.nmea'

positions = {}

# --- Parsing Loop ---
print(f"Opening and parsing {nmea_file_path}...")

with open(nmea_file_path, 'r') as f:
    for line in f:
        # We only care about lines for position
        if line.startswith('$GPGGA') or line.startswith('$GNGGA'):
            try:
                # pynmea2 parses the string into an object
                msg = pynmea2.parse(line)

                # NMEA time is just 'hhmmss.ss'
                # We can store it as a string key
                timestamp = msg.timestamp.strftime('%H%M%S.%f')

                # Store the data
                positions[timestamp] = {
                    'lat': msg.latitude,
                    'lon': msg.longitude,
                    'altitude': msg.altitude
                }

            except pynmea2.ParseError as e:
                # Skip any lines that are corrupted
                pass

print(f"Successfully parsed {len(positions)} positions.")

# --- See the results ---
# Get one of the timestamps to show as an example
if positions:
    example_ts = list(positions.keys())[0]
    print(f"\nExample data for timestamp: {example_ts}")
    print(positions[example_ts])

    # Here's what was parsed from your example line:
    my_line = "$GPGGA,175636.00,3737.5875922,N,12225.5447063,W,1,16,1.1,42.18,M,-32.65,M,,*6E"
    parsed_example = pynmea2.parse(my_line)
    print("\n--- Parsing Your Example Line ---")
    print(f"Original Lat: {parsed_example.lat},{parsed_example.lat_dir}")
    print(f"Decimal Lat:  {parsed_example.latitude}")
    print(f"Original Lon: {parsed_example.lon},{parsed_example.lon_dir}")
    print(f"Decimal Lon:  {parsed_example.longitude}")

Opening and parsing SPAN_Pixel4XL_10Hz.nmea...
Successfully parsed 28531 positions.

Example data for timestamp: 175636.000000
{'lat': 37.62645987, 'lon': -122.425745105, 'altitude': 42.18}

--- Parsing Your Example Line ---
Original Lat: 3737.5875922,N
Decimal Lat:  37.62645987
Original Lon: 12225.5447063,W
Decimal Lon:  -122.425745105


In [43]:
positions

[{'nmea_time': datetime.time(17, 56, 36, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -122.425745105,
  'altitude': 42.18},
 {'nmea_time': datetime.time(17, 56, 36, 100000, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -122.425745105,
  'altitude': 42.18},
 {'nmea_time': datetime.time(17, 56, 36, 200000, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -122.425745105,
  'altitude': 42.18},
 {'nmea_time': datetime.time(17, 56, 36, 300000, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -122.425745105,
  'altitude': 42.18},
 {'nmea_time': datetime.time(17, 56, 36, 400000, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -122.425745105,
  'altitude': 42.18},
 {'nmea_time': datetime.time(17, 56, 36, 500000, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -122.425745105,
  'altitude': 42.18},
 {'nmea_time': datetime.time(17, 56, 36, 600000, tzinfo=datetime.timezone.utc),
  'lat': 37.62645987,
  'lon': -12

### 0.3: Data points alignment

In [14]:
# Convert GNSS data utcTimeMillis to time-of-day
gps_data['utcTimeMillis_numeric'] = pd.to_numeric(gps_data['utcTimeMillis'], errors='coerce')
gps_data['time_of_day'] = pd.to_datetime(gps_data['utcTimeMillis_numeric'], unit='ms', utc=True)

# Convert ground truth list to DataFrame
gt_df = pd.DataFrame(positions)


# Convert nmea_time (already datetime.time objects) to seconds since midnight
def time_to_seconds(t):
    return t.hour * 3600 + t.minute * 60 + t.second + t.microsecond / 1e6


gt_df['seconds_of_day'] = gt_df['nmea_time'].apply(time_to_seconds)

# Convert GNSS time to seconds since midnight
gps_data['seconds_of_day'] = (
        gps_data['time_of_day'].dt.hour * 3600 +
        gps_data['time_of_day'].dt.minute * 60 +
        gps_data['time_of_day'].dt.second +
        gps_data['time_of_day'].dt.microsecond / 1e6
)

# Sort and merge
gps_data = gps_data.sort_values('seconds_of_day')
gt_df = gt_df.sort_values('seconds_of_day')

aligned_data = pd.merge_asof(
    gps_data,
    gt_df[['seconds_of_day', 'lat', 'lon', 'altitude']],
    on='seconds_of_day',
    direction='nearest',
    tolerance=0.1  # 100ms tolerance
)

# Remove rows without ground truth match
aligned_data = aligned_data.dropna(subset=['lat', 'lon'])

aligned_data.head(5)


,Raw,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,...,ChipsetElapsedRealtimeNanos,TimeSeconds,TimeRelative,utcTimeMillis_numeric,time_of_day,time_key,seconds_of_day,lat,lon,altitude
0,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.42567,41.94
1,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.42567,41.94
2,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.42567,41.94
3,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.42567,41.94
4,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.42567,41.94


## 1: Transformation into bicomplex vectors

Using bicomplex number to process signals is a way to naturally isolate the L1 and L5 channels of a compound signal. This is done because each channel is affected by the ionosphere differently. So since noise is introduced into each waveform independently, each waveform should be treated independently.

L1 and L5 channels can be extracted from the imported dataset with the following features:

**Signal Strength ($\frac{C}{N_0}$)**: This stands for the carrier-to-noise density ratio. It represents the ratio between the power of the carrier wave ($C$) and the noise power density ($N_0$). It determines the amplitude ($A$) of the carrier wave.
- **How it works**: The receiver correlates the incoming signal with a local replica. The "height" of the correlation peak determines the signal power.

**Carrier Phase ($\Phi$)**: Measurement of the alignment between the carrier's oscillator wave relative to the satellite's signal.
- **How it works**: The satelite sends a sine wave, the receiver locks on to it. It can measure the fractional part of the wave ($[0, 2\pi]$) with milimiter precision. The receiver knows the fractional part, but does not know how many billions of cyles ($N$) ocurred between the satelite and the phone.
- **Equation**: $$\Phi = \rho + c(dt_r - dt_s) + N\lambda - I + T + \epsilon$$

Where:
- $\rho$: Euclidean distance between the satellite's attena and the receiver's attena.
- $c$: Speed of light ($\approx 299,792,458$ m/s).
- $dt_r$: Receiver clock drift.
- $dt_s$: Satellite clock drift.
- $N$: Number of full sine wave cycles between the satellite and the receiver.
- $\lambda$: Length of one full wave cycle (L1 = 19.03cm, L5 = 25.48cm).
- $I$: "Delay" caused by free electrons in the ionosphere. For carrier phase, this in fact advances the signal instead of delaying it.
- $T$: Delay caused by the lower atmosphere.
- $\epsilon$: Random noise (signal bounce, heat, etc).

**Accumulated Delta Range**: Continuous tracking of the carrier phase converted into meters.
- **How it works**: Instead of resetting the phase every cycle ($0 \dots 2\pi$), the receiver accumulates the total number of cycles it has observed since it started tracking.
- **Equation**: $$\text{ADR} = \phi_{\text{total}} \times \lambda$$

**Carrier Frequency Offset (Doppler Shift + Drift)**: This is the difference between the nominal frequency (e.g., 1575.42 MHz for GPS L1) and the actual frequency received at the phone. This is the time derivative of the phase. If the phase is position, the Frequency Offset is velocity.
- **Equation**: $$f_{\text{offset}} = -\frac{1}{\lambda} \frac{d(\text{ADR})}{dt}$$

These variables can be used to construct the bicomplex form:
$$z_{L1} = \sqrt{10^{\frac{CN0_{L1}}{10}}} \cdot e^{\mathbf{i} \left( \frac{2\pi \cdot \text{ADR}_{L1}}{\lambda_{L1}} \right)}$$
$$z_{L5} = \sqrt{10^{\frac{CN0_{L5}}{10}}} \cdot e^{\mathbf{i} \left( \frac{2\pi \cdot \text{ADR}_{L5}}{\lambda_{L5}} \right)}$$

Resulting in
$$\zeta = z_{L1} + z_{l5} = \left[ A_{L1} \cos(\phi_{L1}) + \mathbf{i} A_{L1} \sin(\phi_{L1}) \right] + \mathbf{j} \left[ A_{L5} \cos(\phi_{L5}) + \mathbf{i} A_{L5} \sin(\phi_{L5}) \right]$$

Which, in its idempotent form, looks like:
$$\zeta = \beta_1 e + \beta_2 e^\dagger$$

This representation allows us to separate the ionospheric error. Since the Ionosphere affects L1 and L5 differently (frequency-dependent), operations in the bicomplex domain can isolate the "pure" geometric range from the "dispersive" ionospheric delay without complex matrix inversions.

In [21]:
# Constants
c = 299792458.0  # Speed of light

# Calculate Wavelength (lambda)
aligned_data['lambda'] = c / aligned_data['CarrierFrequencyHz']

# Calculate Phase (phi) in Radians
# Phase = (ADR_meters / lambda) * 2*pi
aligned_data['phi_rad'] = (aligned_data['AccumulatedDeltaRangeMeters'] / aligned_data['lambda']) * 2 * np.pi

# Calculate Linear Amplitude (A)
# Convert dB-Hz to linear voltage scale
aligned_data['amplitude'] = np.sqrt(10 ** (aligned_data['Cn0DbHz'] / 10.0))

# Create Complex Signal (z)
# z = A * e^(j * phi)
aligned_data['z_complex'] = aligned_data['amplitude'] * np.exp(1j * aligned_data['phi_rad'])

In [13]:
sum(aligned_data['CarrierPhase'].isna())

0

In [18]:
aligned_data[aligned_data['CarrierFrequencyHz'] >= 1.57542e9]

,Raw,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,...,ChipsetElapsedRealtimeNanos,TimeSeconds,TimeRelative,utcTimeMillis_numeric,time_of_day,time_key,seconds_of_day,lat,lon,altitude
0,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.425670,41.94
1,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.425670,41.94
2,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.425670,41.94
3,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.425670,41.94
4,Raw,1599242349431,4514908000000,,,-1283273052523582879,0.1719512939453125,28.230029784026556,-1.690072773787229,13.510000210902188,...,,4514.908,0.0,1599242349431,2020-09-04 17:59:09.431000+00:00,175909.431000,64749.431,37.626536,-122.425670,41.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16539,Raw,1599243607431,5772908000000,,,-1283273052523571637,0.6510372161865234,17.428053979529068,53.75118104102667,11.538163745474398,...,,5772.908,1258.0,1599243607431,2020-09-04 18:20:07.431000+00:00,182007.431000,66007.431,37.442639,-122.252876,138.62
16540,Raw,1599243607431,5772908000000,,,-1283273052523571637,0.6510372161865234,17.428053979529068,53.75118104102667,11.538163745474398,...,,5772.908,1258.0,1599243607431,2020-09-04 18:20:07.431000+00:00,182007.431000,66007.431,37.442639,-122.252876,138.62
16541,Raw,1599243607431,5772908000000,,,-1283273052523571637,0.6510372161865234,17.428053979529068,53.75118104102667,11.538163745474398,...,,5772.908,1258.0,1599243607431,2020-09-04 18:20:07.431000+00:00,182007.431000,66007.431,37.442639,-122.252876,138.62
16542,Raw,1599243607431,5772908000000,,,-1283273052523571637,0.6510372161865234,17.428053979529068,53.75118104102667,11.538163745474398,...,,5772.908,1258.0,1599243607431,2020-09-04 18:20:07.431000+00:00,182007.431000,66007.431,37.442639,-122.252876,138.62


In [22]:
aligned_data.columns

Index(['Raw', 'utcTimeMillis', 'TimeNanos', 'LeapSecond',
       'TimeUncertaintyNanos', 'FullBiasNanos', 'BiasNanos',
       'BiasUncertaintyNanos', 'DriftNanosPerSecond',
       'DriftUncertaintyNanosPerSecond', 'HardwareClockDiscontinuityCount',
       'Svid', 'TimeOffsetNanos', 'State', 'ReceivedSvTimeNanos',
       'ReceivedSvTimeUncertaintyNanos', 'Cn0DbHz',
       'PseudorangeRateMetersPerSecond',
       'PseudorangeRateUncertaintyMetersPerSecond',
       'AccumulatedDeltaRangeState', 'AccumulatedDeltaRangeMeters',
       'AccumulatedDeltaRangeUncertaintyMeters', 'CarrierFrequencyHz',
       'CarrierCycles', 'CarrierPhase', 'CarrierPhaseUncertainty',
       'MultipathIndicator', 'SnrInDb', 'ConstellationType', 'AgcDb',
       'BasebandCn0DbHz', 'FullInterSignalBiasNanos',
       'FullInterSignalBiasUncertaintyNanos', 'SatelliteInterSignalBiasNanos',
       'SatelliteInterSignalBiasUncertaintyNanos', 'CodeType',
       'ChipsetElapsedRealtimeNanos', 'TimeSeconds', 'TimeRelative',

In [23]:
# Define Frequency Masks
# L1 is approx 1575.42 MHz, L5 is approx 1176.45 MHz
mask_l1 = np.abs(aligned_data['CarrierFrequencyHz'] - 1.57542e9) < 1e7
mask_l5 = np.abs(aligned_data['CarrierFrequencyHz'] - 1.17645e9) < 1e7

# Create the L1 "Anchor" DataFrame
cols_l1 = [
    'TimeNanos', 'Svid', 'ConstellationType',  # Join Keys
    'lat', 'lon', 'altitude',                  # Ground Truth
    'z_complex', 'Cn0DbHz', 'AccumulatedDeltaRangeMeters' # L1 Signal Data
]
df_l1 = aligned_data.loc[mask_l1, cols_l1].copy()

# Rename L1 specific columns to avoid confusion later
df_l1 = df_l1.rename(columns={
    'z_complex': 'z_L1',
    'Cn0DbHz': 'Cn0_L1',
    'AccumulatedDeltaRangeMeters': 'ADR_L1'
})

# Create the L5 "Augmentation" DataFrame
# We exclude 'lat', 'lon' here to avoid duplicates.
cols_l5 = [
    'TimeNanos', 'Svid', 'ConstellationType',  # Join Keys
    'z_complex', 'Cn0DbHz', 'AccumulatedDeltaRangeMeters' # L5 Signal Data
]
df_l5 = aligned_data.loc[mask_l5, cols_l5].copy()

# Rename L5 specific columns
df_l5 = df_l5.rename(columns={
    'z_complex': 'z_L5',
    'Cn0DbHz': 'Cn0_L5',
    'AccumulatedDeltaRangeMeters': 'ADR_L5'
})

# Pivot the table
# This automatically aligns L1 and L5 based on Time and Satellite.
bicomplex_df = pd.merge(
    df_l1,
    df_l5,
    on=['TimeNanos', 'Svid', 'ConstellationType'],
    how='inner' # Use 'inner' to keep only Valid Pairs, or 'left' to keep all L1s
)

       TimeNanos  Svid  ConstellationType        lat        lon  altitude  \
0  4514908000000    25                  1  37.626536 -122.42567     41.94   
1  4514908000000    26                  1  37.626536 -122.42567     41.94   
2  4515908000000    18                  1  37.626536 -122.42567     41.94   
3  4515908000000    25                  1  37.626536 -122.42567     41.94   
4  4515908000000    26                  1  37.626536 -122.42567     41.94   

                   z_L1  Cn0_L1         ADR_L1                  z_L5  Cn0_L5  \
0  -3.600548+64.464951j    36.2   60174.332742  -5.067047-30.484711j    29.8   
1 -23.962618- 1.110348j    27.6   -6355.712001   8.826782-22.009754j    27.5   
2   0.892832-64.559249j    36.2 -738655.180750   5.559043+ 0.000000j    14.9   
3 -43.673709+63.095512j    37.7   60856.552194  22.439134-38.623148j    33.0   
4   9.975946+19.188946j    26.7   -6500.398686 -16.240740-24.641468j    29.4   

         ADR_L5  
0  1.861561e+06  
1 -9.541315e+05  
2 

In [24]:
bicomplex_df.head()

,TimeNanos,Svid,ConstellationType,lat,lon,altitude,z_L1,Cn0_L1,ADR_L1,z_L5,Cn0_L5,ADR_L5
0,4514908000000,25,1,37.626536,-122.42567,41.94,-3.600548+64.464951j,36.2,60174.332742,-5.067047-30.484711j,29.8,1.861561e+06
1,4514908000000,26,1,37.626536,-122.42567,41.94,-23.962618- 1.110348j,27.6,-6355.712001,8.826782-22.009754j,27.5,-9.541315e+05
2,4515908000000,18,1,37.626536,-122.42567,41.94,0.892832-64.559249j,36.2,-738655.180750,5.559043+ 0.000000j,14.9,0.000000e+00
3,4515908000000,25,1,37.626536,-122.42567,41.94,-43.673709+63.095512j,37.7,60856.552194,22.439134-38.623148j,33.0,1.862244e+06
4,4515908000000,26,1,37.626536,-122.42567,41.94,9.975946+19.188946j,26.7,-6500.398686,-16.240740-24.641468j,29.4,-9.542763e+05


## 2: Experiments
The purpose of this notebook is to test the performance of neural networks based on bicomplex algebra, against neural networks based on other algebras, namely:
- Real valued neural networks
- Complex valued neural networks
- Quaternion valued neural networks

Since L1 readings vastly outnumber L5 readings, the experiments will be run on different versions of the previously transformed dataset, namely:
- Full dataset including entries wich have only L1 channel
- Filtered dataset for entries that have both L1 and L5 channels